In [3]:
import tifffile as tf
import napari
import zarr
import dask.array as da
import pandas as pd
import numpy as np
import umap
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import cellcutter
# import closest
# import forcelayout as fl
# import gridify




# image_path = "/Volumes/Simon/Greg/WD-76845-097.ome.tif"
# segmentation_path = "/Volumes/Simon/Greg/segmentation_mask/WD-76845-097.tif"
csv_path = "/Users/swarchol/Research/bed/data/for_simon/unmicst-WD-76845-097_cellRing.csv"
# dest_path = "/Volumes/Simon/Greg/cut"



In [4]:
csv_df = pd.read_csv(csv_path)

In [10]:
csv_df

,CellID,Hoechst0,AF488,AF555,AF647,Hoechst1,A488,A555,A647,Hoechst2,...,Y_centroid,column_centroid,row_centroid,Area,MajorAxisLength,MinorAxisLength,Eccentricity,Solidity,Extent,Orientation
0,1,362.993151,828.267123,343.315068,652.842466,323.547945,784.883562,355.349315,756.650685,721.013699,...,16.794521,17926.664384,16.794521,146,19.757154,9.911892,0.865050,0.935897,0.506944,-0.964550
1,2,368.467456,880.147929,378.136095,678.573964,342.822485,796.621302,365.402367,759.100592,356.786982,...,543.976331,19121.662722,543.976331,169,16.149986,13.456914,0.552902,0.960227,0.754464,-1.192025
2,3,376.638060,919.145522,379.876866,685.891791,357.992537,861.473881,377.175373,774.276119,375.753731,...,827.253731,20627.615672,827.253731,268,25.022285,14.012230,0.828499,0.927336,0.620370,-0.235120
3,4,380.475610,940.457317,381.432927,686.719512,357.817073,864.149390,381.128049,765.963415,373.009146,...,837.073171,20611.920732,837.073171,328,35.875258,13.288510,0.928869,0.750572,0.585714,-0.028089
4,5,377.841837,942.974490,384.709184,685.076531,352.392857,863.056122,374.275510,775.913265,366.795918,...,835.576531,20601.637755,835.576531,196,23.260397,11.497359,0.869298,0.911628,0.560000,-0.103986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1242751,1242752,36016.336842,8849.473684,3033.905263,1129.221053,37950.684211,2214.505263,870.042105,1557.115789,37927.968421,...,26818.968421,7104.242105,26818.968421,95,11.156316,10.868287,0.225763,0.950000,0.785124,0.336061
1242752,1242753,23205.422360,11025.291925,3764.826087,1337.142857,24161.621118,2527.944099,921.006211,1236.534161,21913.875776,...,26821.322981,6942.645963,26821.322981,161,16.165999,12.759782,0.614011,0.964072,0.728507,-1.500372
1242753,1242754,35534.086420,8966.320988,2977.691358,1183.493827,37194.790123,2233.728395,863.283951,1608.209877,37491.049383,...,26822.901235,7010.135802,26822.901235,81,12.012915,8.890512,0.672519,0.987805,0.613636,0.952358
1242754,1242755,34436.795812,10391.340314,3508.162304,1270.408377,37016.062827,2543.979058,919.445026,1343.382199,35432.481675,...,26826.319372,6924.958115,26826.319372,191,17.788878,13.849243,0.627603,0.945545,0.757937,-0.151823


In [7]:
csv_df.columns

# Read parquet file
parquet_path = "/Users/swarchol/Research/bed/data/clustering.parquet"
parquet_df = pd.read_parquet(parquet_path)

In [13]:
# parquet_df
# sort pqrquet_df by cell_id
parquet_df = parquet_df.sort_values(by='CellID')
parquet_df

,CellID,Hoechst0,Hoechst1,Hoechst2,anti_CD3,anti_CD45RO,Hoechst3,Keratin_570,aSMA_660,Hoechst4,...,Eccentricity,Solidity,Extent,Orientation,Sample,Condition,Replicate,emb1,emb2,cluster_2d
798818,47,15160.563636,46937.436364,42258.036364,0.235838,0.446541,24461.163636,0.270726,0.175902,29225.363636,...,0.921447,0.916667,0.763889,-1.345788,WD-76845-097,CRC,1,8.872235,19.100700,-1
77194,49,11740.781250,50723.343750,37632.750000,0.227137,0.584889,20075.703125,0.307917,0.174437,23681.062500,...,0.881066,0.888889,0.653061,-1.338040,WD-76845-097,CRC,1,8.379912,20.137026,48
37786,51,9565.513514,35690.567568,27032.162162,0.200757,0.436587,14392.621622,0.223040,0.162788,16623.810811,...,0.745784,0.902439,0.685185,-1.475095,WD-76845-097,CRC,1,8.799885,19.362713,-1
37252,52,7308.113636,24641.750000,19214.886364,0.228204,0.662923,9205.522727,0.272994,0.207245,11968.636364,...,0.825548,0.936170,0.571429,-1.390750,WD-76845-097,CRC,1,8.504932,20.219538,48
631268,53,19249.623188,57667.202899,53509.811594,0.261224,0.704192,30691.275362,0.295741,0.296045,35399.927536,...,0.842002,0.907895,0.663462,-1.441929,WD-76845-097,CRC,1,8.576233,20.350969,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38177,1242746,18694.276596,18604.936170,17079.659574,0.248739,0.205520,6982.680851,0.136490,0.334573,9554.489362,...,0.618630,0.959184,0.839286,-0.935988,WD-76845-097,CRC,1,18.186203,2.827556,72
553647,1242748,31240.095238,34545.301587,32678.349206,0.361773,0.381520,12513.238095,0.219673,0.126889,17597.158730,...,0.828981,0.940299,0.583333,0.459917,WD-76845-097,CRC,1,11.654655,19.720819,-1
315309,1242751,45004.015873,47513.269841,44139.714286,0.229271,0.467404,16901.253968,0.152905,0.283555,20190.285714,...,0.847186,0.954545,0.630000,0.830085,WD-76845-097,CRC,1,18.262630,3.071928,72
728644,1242753,33568.983607,35640.344262,31911.131148,0.233328,0.515689,12369.524590,0.134693,0.331565,15530.245902,...,0.753467,0.953125,0.693182,-1.363981,WD-76845-097,CRC,1,19.307392,3.061911,72


In [14]:
# # showq csv_df rows 45-50
csv_df[45:50]

,CellID,Hoechst0,AF488,AF555,AF647,Hoechst1,A488,A555,A647,Hoechst2,...,Y_centroid,column_centroid,row_centroid,Area,MajorAxisLength,MinorAxisLength,Eccentricity,Solidity,Extent,Orientation
45,46,351.851550,847.026101,348.316476,676.399674,320.239804,783.380098,357.494290,739.859706,335.435563,...,1223.076672,18004.513866,1223.076672,613,40.776219,20.059508,0.870628,0.902798,0.528448,0.433260
46,47,10036.628743,5926.628743,2123.449102,911.389222,29461.754491,1931.670659,745.419162,1114.281437,25063.395210,...,1228.341317,9421.479042,1228.341317,167,18.973863,11.252757,0.805154,0.948864,0.773148,-1.325285
47,48,347.486486,851.189189,347.810811,669.243243,313.743243,782.581081,355.378378,747.324324,326.432432,...,1231.864865,18019.081081,1231.864865,74,11.370983,8.824451,0.630671,0.961039,0.569231,-0.176832
48,49,10313.189394,5363.946970,1906.780303,895.659091,40543.757576,1718.219697,695.007576,1000.386364,30388.886364,...,1232.954545,9460.166667,1232.954545,132,16.480047,11.585922,0.711163,0.874172,0.564103,-1.278167
49,50,7110.295455,5181.181818,1835.659091,880.159091,24099.863636,1655.704545,681.454545,1007.136364,17946.409091,...,1234.750000,9468.454545,1234.750000,44,12.086412,4.802855,0.917656,0.880000,0.500000,-0.526671


In [10]:
seg_zarr = zarr.open(zarr_path)

,CellID,Hoechst0,AF488,AF555,AF647,Hoechst1,A488,A555,A647,Hoechst2,...,Y_centroid,column_centroid,row_centroid,Area,MajorAxisLength,MinorAxisLength,Eccentricity,Solidity,Extent,Orientation
0,1,362.993151,828.267123,343.315068,652.842466,323.547945,784.883562,355.349315,756.650685,721.013699,...,16.794521,17926.664384,16.794521,146,19.757154,9.911892,0.865050,0.935897,0.506944,-0.964550
1,2,368.467456,880.147929,378.136095,678.573964,342.822485,796.621302,365.402367,759.100592,356.786982,...,543.976331,19121.662722,543.976331,169,16.149986,13.456914,0.552902,0.960227,0.754464,-1.192025
2,3,376.638060,919.145522,379.876866,685.891791,357.992537,861.473881,377.175373,774.276119,375.753731,...,827.253731,20627.615672,827.253731,268,25.022285,14.012230,0.828499,0.927336,0.620370,-0.235120
3,4,380.475610,940.457317,381.432927,686.719512,357.817073,864.149390,381.128049,765.963415,373.009146,...,837.073171,20611.920732,837.073171,328,35.875258,13.288510,0.928869,0.750572,0.585714,-0.028089
4,5,377.841837,942.974490,384.709184,685.076531,352.392857,863.056122,374.275510,775.913265,366.795918,...,835.576531,20601.637755,835.576531,196,23.260397,11.497359,0.869298,0.911628,0.560000,-0.103986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1242751,1242752,36016.336842,8849.473684,3033.905263,1129.221053,37950.684211,2214.505263,870.042105,1557.115789,37927.968421,...,26818.968421,7104.242105,26818.968421,95,11.156316,10.868287,0.225763,0.950000,0.785124,0.336061
1242752,1242753,23205.422360,11025.291925,3764.826087,1337.142857,24161.621118,2527.944099,921.006211,1236.534161,21913.875776,...,26821.322981,6942.645963,26821.322981,161,16.165999,12.759782,0.614011,0.964072,0.728507,-1.500372
1242753,1242754,35534.086420,8966.320988,2977.691358,1183.493827,37194.790123,2233.728395,863.283951,1608.209877,37491.049383,...,26822.901235,7010.135802,26822.901235,81,12.012915,8.890512,0.672519,0.987805,0.613636,0.952358
1242754,1242755,34436.795812,10391.340314,3508.162304,1270.408377,37016.062827,2543.979058,919.445026,1343.382199,35432.481675,...,26826.319372,6924.958115,26826.319372,191,17.788878,13.849243,0.627603,0.945545,0.757937,-0.151823
